In [1]:
import pandas as pd
import fsspec
from fv3net.diagnostics.prognostic_run import ComputedDiagnosticsList

### Load metrics from ensemble of 10-day forecasts, for 3-7 day RMSE metrics

In [2]:
start_dates = ['0805', '0813', '0821', '0829']
urls_baseline = {
    f'baseline-{ic}': f'gs://vcm-ml-experiments/default/2022-12-12/ml-on-fine-ic-baseline-2016{ic}.000000-trial-0/fv3gfs_run_diagnostics'
    for ic in start_dates
}
urls_ml_physics = {
    f'ml-physics-{ic}': f'gs://vcm-ml-experiments/default/2022-12-12/ml-on-fine-ic-prog-2016{ic}.000000-trial-0/fv3gfs_run_diagnostics'
    for ic in start_dates
}
urls_nudge_to_fine = {
    f'nudge-to-fine-{ic}': f'gs://vcm-ml-experiments/default/2022-12-13/ml-on-fine-ic-n2f-2016{ic}.000000-trial-0/fv3gfs_run_diagnostics'
    for ic in start_dates
}
urls = {**urls_baseline, **urls_ml_physics, **urls_nudge_to_fine}

In [3]:
cd = ComputedDiagnosticsList.from_dict(urls)
metrics = cd.load_metrics()

In [4]:
metrics.types

{'percentile_25',
 'percentile_50',
 'percentile_75',
 'percentile_90',
 'percentile_99',
 'percentile_99.9',
 'rmse_3day',
 'rmse_5day',
 'rmse_days_3to7_avg',
 'rmse_of_time_mean',
 'rmse_of_time_mean_land',
 'rmse_of_time_mean_sea',
 'time_and_global_mean_bias',
 'time_and_global_mean_value',
 'time_and_land_mean_bias',
 'time_and_land_mean_value',
 'time_and_sea_mean_bias',
 'time_and_sea_mean_value',
 'tropical_ascent_region_mean',
 'tropics_max_minus_min'}

In [5]:
def metric_with_improvements(metrics, metric_type, variable):
    output = metrics.get_metric_all_runs(metric_type, variable)
    output.loc[:, 'run_type'] = [x[:-5] for x in output.loc[:, 'run']]

    improvements = []
    for run in output.run:
        corresponding_baseline = 'baseline-' + run[-4:]
        run_value = output.loc[output['run'] == run]['value']
        baseline_value = output.loc[output['run'] == corresponding_baseline]['value']
        improvements.append(run_value.item() - baseline_value.item())
    output['improvement'] = improvements
    
    return output

In [6]:
rmse_h500 = metric_with_improvements(metrics, 'rmse_days_3to7_avg', 'h500')

/home/OliWM/miniconda3/envs/fv3net/lib/python3.8/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
<ipython-input-5-07b9eccbf770>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output['improvement'] = improvements


In [7]:
rmse_h500

,run,baseline,metric,value,units,run_type,improvement
38,baseline-0805,True,rmse_days_3to7_avg/h500,56.206383,m,baseline,0.000000
329,baseline-0813,True,rmse_days_3to7_avg/h500,54.000507,m,baseline,0.000000
620,baseline-0821,True,rmse_days_3to7_avg/h500,63.744751,m,baseline,0.000000
911,baseline-0829,True,rmse_days_3to7_avg/h500,55.898529,m,baseline,0.000000
1202,ml-physics-0805,False,rmse_days_3to7_avg/h500,56.710182,m,ml-physics,0.503799
1493,ml-physics-0813,False,rmse_days_3to7_avg/h500,56.753613,m,ml-physics,2.753105
1784,ml-physics-0821,False,rmse_days_3to7_avg/h500,63.387745,m,ml-physics,-0.357006
2075,ml-physics-0829,False,rmse_days_3to7_avg/h500,56.123924,m,ml-physics,0.225395
2366,nudge-to-fine-0805,False,rmse_days_3to7_avg/h500,53.573738,m,nudge-to-fine,-2.632645
2657,nudge-to-fine-0813,False,rmse_days_3to7_avg/h500,52.510609,m,nudge-to-fine,-1.489899


#### Mean

In [8]:
rmse_h500.groupby('run_type').mean()

,baseline,value,improvement
run_type,,,
baseline,1.0,57.462543,0.000000
ml-physics,0.0,58.243866,0.781323
nudge-to-fine,0.0,55.980771,-1.481771


#### Standard deviation

In [9]:
rmse_h500.groupby('run_type').std()

,baseline,value,improvement
run_type,,,
baseline,0.0,4.300229,0.000000
ml-physics,0.0,3.441254,1.362570
nudge-to-fine,0.0,4.932759,0.887899


In [10]:
rmse_tmp850 = metric_with_improvements(metrics, 'rmse_days_3to7_avg', 'tmp850')

/home/OliWM/miniconda3/envs/fv3net/lib/python3.8/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
<ipython-input-5-07b9eccbf770>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output['improvement'] = improvements


In [11]:
rmse_tmp850.groupby('run_type').mean()

,baseline,value,improvement
run_type,,,
baseline,1.0,2.867559,0.000000
ml-physics,0.0,2.842562,-0.024997
nudge-to-fine,0.0,2.590904,-0.276655


In [12]:
rmse_tmp850.groupby('run_type').std()

,baseline,value,improvement
run_type,,,
baseline,0.0,0.108910,0.000000
ml-physics,0.0,0.087061,0.032372
nudge-to-fine,0.0,0.121419,0.016130


### Load metrics from 35-day forecasts, for time-mean metrics

In [13]:
urls_35day = {
    "baseline": "gs://vcm-ml-experiments/default/2022-11-21/ml-on-fine-baseline-long-trial-0/fv3gfs_run_diagnostics",
    "ML-physics": f"gs://vcm-ml-experiments/default/2022-11-18/ml-on-fine-ensemble-prog-5-long-trial-0/fv3gfs_run_diagnostics",
    "nudge-to-fine": "gs://vcm-ml-experiments/default/2022-05-09/ml-on-fine-hybrid-comp-n2f-v3/fv3gfs_run_diagnostics",
}

In [14]:
cd = ComputedDiagnosticsList.from_dict(urls_35day)
metrics_35day = cd.load_metrics()

In [15]:
metrics_35day.get_metric_all_runs('rmse_of_time_mean', 'total_precip_to_surface')

,run,baseline,metric,value,units
183,baseline,True,rmse_of_time_mean/total_precip_to_surface,3.246038,mm/day
474,ML-physics,False,rmse_of_time_mean/total_precip_to_surface,2.847433,mm/day
765,nudge-to-fine,False,rmse_of_time_mean/total_precip_to_surface,2.551957,mm/day


In [16]:
metrics_35day.get_metric_all_runs('time_and_land_mean_bias', 'total_precip_to_surface')

,run,baseline,metric,value,units
145,baseline,True,time_and_land_mean_bias/total_precip_to_surface,0.950176,mm/day
436,ML-physics,False,time_and_land_mean_bias/total_precip_to_surface,0.778978,mm/day
727,nudge-to-fine,False,time_and_land_mean_bias/total_precip_to_surface,0.686899,mm/day


In [17]:
metrics_35day.get_metric_all_runs('rmse_of_time_mean', 'tmp850')

,run,baseline,metric,value,units
181,baseline,True,rmse_of_time_mean/tmp850,2.064156,K
472,ML-physics,False,rmse_of_time_mean/tmp850,1.827706,K
763,nudge-to-fine,False,rmse_of_time_mean/tmp850,1.960721,K


In [18]:
metrics_35day.get_metric_all_runs('rmse_of_time_mean', 'tmp200')

,run,baseline,metric,value,units
180,baseline,True,rmse_of_time_mean/tmp200,2.084215,K
471,ML-physics,False,rmse_of_time_mean/tmp200,2.176124,K
762,nudge-to-fine,False,rmse_of_time_mean/tmp200,2.166827,K
